In [3]:


import os
import json
import random
from typing import List, Dict, Tuple
from dataclasses import dataclass
from enum import Enum
import time

# Try to import OpenAI, but provide fallback
try:
    import openai
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("OpenAI library not available. Running in mock mode.")

class GenerationMode(Enum):
    MOCK = "mock"
    OPENAI = "openai"

@dataclass
class CampaignBrief:
    product_name: str
    target_audience: str
    key_features: List[str]
    brand_voice: str
    campaign_goal: str
    competitors: List[str] = None

    def __post_init__(self):
        if self.competitors is None:
            self.competitors = []

@dataclass
class AdCopy:
    headline: str
    body: str
    call_to_action: str
    tone: str
    target_segment: str

@dataclass
class AdVariation:
    copy: AdCopy
    image_prompt: str
    generated_image: str = None
    coherence_score: float = 0.0

class UserInputHandler:
    """Handles all user inputs for campaign creation"""

    @staticmethod
    def get_campaign_brief() -> CampaignBrief:
        """Get complete campaign brief from user"""
        print("\n" + "="*50)
        print("📝 AD CAMPAIGN BRIEF CREATION")
        print("="*50)

        # Basic product info
        product_name = input("\n🎯 Product Name: ").strip()
        while not product_name:
            print("❌ Product name is required!")
            product_name = input("🎯 Product Name: ").strip()

        target_audience = input("👥 Target Audience (e.g., 'Young professionals 25-35'): ").strip()
        while not target_audience:
            print("❌ Target audience is required!")
            target_audience = input("👥 Target Audience: ").strip()

        # Key features
        print("\n🔑 Key Features (Enter one per line, press Enter twice to finish):")
        key_features = []
        while True:
            feature = input("  → Feature: ").strip()
            if feature:
                key_features.append(feature)
            elif key_features:  # Empty input but we have at least one feature
                break
            else:
                print("  ❌ Please enter at least one key feature")

        # Brand voice
        brand_voice = input("\n🎭 Brand Voice (e.g., 'Professional, Friendly, Innovative'): ").strip()
        while not brand_voice:
            print("❌ Brand voice is required!")
            brand_voice = input("🎭 Brand Voice: ").strip()

        # Campaign goal
        campaign_goal = input("\n🎯 Campaign Goal (e.g., 'Increase sales by 25%'): ").strip()
        while not campaign_goal:
            print("❌ Campaign goal is required!")
            campaign_goal = input("🎯 Campaign Goal: ").strip()

        # Competitors (optional)
        print("\n🏢 Competitors (Enter one per line, press Enter twice to skip):")
        competitors = []
        while True:
            competitor = input("  → Competitor: ").strip()
            if competitor:
                competitors.append(competitor)
            else:
                break

        return CampaignBrief(
            product_name=product_name,
            target_audience=target_audience,
            key_features=key_features,
            brand_voice=brand_voice,
            campaign_goal=campaign_goal,
            competitors=competitors if competitors else None
        )

    @staticmethod
    def get_generation_settings() -> tuple:
        """Get number of variations and generation mode from user"""
        print("\n" + "="*50)
        print("⚙️  GENERATION SETTINGS")
        print("="*50)

        # Number of variations
        while True:
            try:
                num_variations = int(input("\n🔢 How many ad variations to generate? (3-10): "))
                if 3 <= num_variations <= 10:
                    break
                else:
                    print("❌ Please enter a number between 3 and 10")
            except ValueError:
                print("❌ Please enter a valid number")

        # Generation mode
        print("\n🤖 Choose Generation Mode:")
        print("1. MOCK Mode (Fast, sample data - No API keys needed)")
        print("2. OPENAI Mode (Real AI - Requires OpenAI API key)")

        while True:
            mode_choice = input("\n🎛️  Enter your choice (1 or 2): ").strip()
            if mode_choice == "1":
                mode = GenerationMode.MOCK
                break
            elif mode_choice == "2":
                if not OPENAI_AVAILABLE:
                    print("❌ OpenAI library not installed. Install with: pip install openai")
                    print("🔄 Using MOCK mode instead")
                    mode = GenerationMode.MOCK
                else:
                    mode = GenerationMode.OPENAI
                break
            else:
                print("❌ Please enter 1 or 2")

        return num_variations, mode

    @staticmethod
    def get_openai_key() -> str:
        """Get OpenAI API key from user"""
        print("\n🔑 OpenAI API Key:")
        print("Note: You can get your API key from https://platform.openai.com/account/api-keys")

        api_key = input("Enter your OpenAI API key: ").strip()
        while not api_key:
            print("❌ API key is required for OpenAI mode!")
            api_key = input("Enter your OpenAI API key: ").strip()

        return api_key

class AdCampaignGenerator:
    def __init__(self, openai_api_key: str = None, mode: GenerationMode = GenerationMode.MOCK):
        self.mode = mode

        if mode == GenerationMode.OPENAI:
            if not OPENAI_AVAILABLE:
                raise ImportError("OpenAI library not installed. Run: pip install openai")
            if not openai_api_key:
                raise ValueError("Please provide a valid OpenAI API key")

            self.openai_client = openai.OpenAI(api_key=openai_api_key)
        else:
            print("🎭 Running in MOCK mode - generating sample data without API calls")

    def generate_ad_copies(self, brief: CampaignBrief, num_variations: int = 5) -> List[AdCopy]:
        """Generate multiple ad copy variations based on campaign brief"""

        if self.mode == GenerationMode.MOCK:
            return self._generate_mock_copies(brief, num_variations)

        # OpenAI implementation
        prompt = f"""
        Create {num_variations} distinct ad copy variations for the following campaign:

        PRODUCT: {brief.product_name}
        TARGET AUDIENCE: {brief.target_audience}
        KEY FEATURES: {', '.join(brief.key_features)}
        BRAND VOICE: {brief.brand_voice}
        GOAL: {brief.campaign_goal}
        COMPETITORS: {', '.join(brief.competitors) if brief.competitors else 'None specified'}

        For each variation, provide JSON with:
        - headline (attention-grabbing, 5-8 words)
        - body (compelling description, 20-30 words)
        - call_to_action (action-oriented, 3-5 words)
        - tone (e.g., professional, casual, inspirational)
        - target_segment (specific subset of audience)

        Return ONLY valid JSON array.
        """

        try:
            response = self.openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.8,
                max_tokens=1000
            )

            content = response.choices[0].message.content.strip()
            # Clean the response to ensure valid JSON
            if content.startswith("```json"):
                content = content[7:]
            if content.endswith("```"):
                content = content[:-3]
            content = content.strip()

            copies_data = json.loads(content)
            ad_copies = []
            for copy_data in copies_data:
                ad_copies.append(AdCopy(
                    headline=copy_data['headline'],
                    body=copy_data['body'],
                    call_to_action=copy_data['call_to_action'],
                    tone=copy_data['tone'],
                    target_segment=copy_data['target_segment']
                ))
            return ad_copies

        except Exception as e:
            print(f"❌ OpenAI copy generation failed: {e}. Falling back to mock data.")
            return self._generate_mock_copies(brief, num_variations)

    def _generate_mock_copies(self, brief: CampaignBrief, num_variations: int) -> List[AdCopy]:
        """Generate realistic mock ad copies"""

        tones = ['Professional', 'Casual', 'Inspirational', 'Urgent', 'Friendly', 'Luxury']
        segments = [
            f"Busy {brief.target_audience.split(' ')[0]}",
            "Quality-focused buyers",
            "Early adopters",
            "Value-conscious consumers",
            "Lifestyle enthusiasts"
        ]

        ctas = ["Shop Now", "Learn More", "Get Started", "Buy Today", "Discover How", "Try Free"]

        mock_copies = []
        for i in range(num_variations):
            tone = random.choice(tones)
            segment = random.choice(segments)
            feature = random.choice(brief.key_features)

            headline_variations = [
                f"Revolutionary {brief.product_name} - Transform Your Life",
                f"Discover {brief.product_name}'s {feature} Feature",
                f"Why {brief.target_audience.split(' ')[0]} Love {brief.product_name}",
                f"The Future of {brief.product_name.split(' ')[-1]} Is Here",
                f"Experience {brief.product_name} - Unlike Any Other"
            ]

            body_variations = [
                f"Experience the difference with {brief.product_name}. Perfect for {brief.target_audience} who value quality and innovation.",
                f"Join thousands of satisfied customers using {brief.product_name}. {feature} ensures you get the best experience.",
                f"Transform your daily routine with {brief.product_name}. Designed specifically for {brief.target_audience} needs.",
                f"Discover why {brief.product_name} stands out from competitors. {random.choice(brief.key_features)} makes all the difference."
            ]

            mock_copies.append(AdCopy(
                headline=random.choice(headline_variations),
                body=random.choice(body_variations),
                call_to_action=random.choice(ctas),
                tone=tone,
                target_segment=segment
            ))

        return mock_copies

    def create_image_prompts(self, ad_copies: List[AdCopy]) -> List[str]:
        """Convert ad copies into detailed image generation prompts"""

        if self.mode == GenerationMode.MOCK:
            return self._generate_mock_prompts(ad_copies)

        # OpenAI implementation
        image_prompts = []
        for copy in ad_copies:
            prompt = f"""
            Create a detailed image generation prompt for an advertisement with:
            - Headline: {copy.headline}
            - Description: {copy.body}
            - Tone: {copy.tone}
            - Target: {copy.target_segment}

            Make it: professional photography, relevant to message, visually appealing.
            Provide only the image prompt text.
            """

            try:
                response = self.openai_client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.7,
                    max_tokens=150
                )

                image_prompt = response.choices[0].message.content.strip()
                image_prompts.append(image_prompt)

            except Exception as e:
                print(f"❌ Prompt generation failed: {e}")
                image_prompts.append(self._create_mock_prompt(copy))

        return image_prompts

    def _generate_mock_prompts(self, ad_copies: List[AdCopy]) -> List[str]:
        """Generate mock image prompts"""
        mock_prompts = []
        for copy in ad_copies:
            mock_prompts.append(self._create_mock_prompt(copy))
        return mock_prompts

    def _create_mock_prompt(self, copy: AdCopy) -> str:
        """Create a single mock image prompt"""
        settings = ["modern office", "cozy home", "outdoor adventure", "professional studio", "urban landscape"]
        styles = ["professional photography", "lifestyle shot", "product focus", "contextual scene"]

        return f"{copy.tone} style image of {copy.target_segment} using {copy.headline.split(' ')[0]} in a {random.choice(settings)}, {random.choice(styles)}, high quality, professional lighting, advertising aesthetic"

    def generate_images(self, image_prompts: List[str]) -> List[str]:
        """Generate images - mock implementation with placeholder images"""

        print("🖼️  Image generation would normally call DALL-E API here")
        print("📸 Using placeholder images for demonstration")

        # Using placeholder images for demo
        placeholder_urls = [
            "https://via.placeholder.com/1024x1024/4A90E2/FFFFFF?text=Ad+Image+1",
            "https://via.placeholder.com/1024x1024/50E3C2/FFFFFF?text=Ad+Image+2",
            "https://via.placeholder.com/1024x1024/B8E986/FFFFFF?text=Ad+Image+3",
            "https://via.placeholder.com/1024x1024/F5A623/FFFFFF?text=Ad+Image+4",
            "https://via.placeholder.com/1024x1024/BD10E0/FFFFFF?text=Ad+Image+5"
        ]

        # Cycle through placeholder URLs
        return [random.choice(placeholder_urls) for _ in range(len(image_prompts))]

    def evaluate_coherence(self, variations: List[AdVariation]) -> List[AdVariation]:
        """Evaluate how well images match their corresponding ad copies"""

        for variation in variations:
            if self.mode == GenerationMode.MOCK:
                # Generate realistic mock scores
                base_score = random.uniform(6.0, 9.5)
                # Add some logic to make scores more meaningful
                if "professional" in variation.image_prompt.lower() and "professional" in variation.copy.tone.lower():
                    base_score += 1.0
                variation.coherence_score = min(10.0, base_score)
            else:
                # OpenAI evaluation implementation
                evaluation_prompt = f"""
                Rate coherence between ad copy and image (1-10):

                AD: {variation.copy.headline} - {variation.copy.body}
                IMAGE: {variation.image_prompt}

                Consider: relevance, emotional alignment, audience appeal.
                Return only a number.
                """

                try:
                    response = self.openai_client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[{"role": "user", "content": evaluation_prompt}],
                        temperature=0.3,
                        max_tokens=10
                    )

                    score_text = response.choices[0].message.content.strip()
                    score = float(score_text)
                    variation.coherence_score = min(10, max(1, score))

                except Exception as e:
                    print(f"❌ Coherence evaluation failed: {e}")
                    variation.coherence_score = random.uniform(5.0, 8.0)

        return sorted(variations, key=lambda x: x.coherence_score, reverse=True)

    def create_ab_test_recommendations(self, ranked_variations: List[AdVariation]) -> Dict:
        """Create A/B testing recommendations based on variations"""

        # Group by target segment for testing
        segments = {}
        for variation in ranked_variations:
            segment = variation.copy.target_segment
            if segment not in segments:
                segments[segment] = []
            segments[segment].append(variation)

        test_recommendations = {}
        for segment, variations in segments.items():
            # Take top 2-3 variations per segment for testing
            top_variations = variations[:min(3, len(variations))]
            test_recommendations[segment] = {
                'variations': top_variations,
                'testing_priority': 'High' if len(top_variations) >= 2 else 'Medium',
                'sample_size_recommendation': self._calculate_sample_size(len(top_variations)),
                'key_metrics': ['CTR', 'Conversion Rate', 'Engagement Rate'],
                'recommended_duration': f"{len(top_variations) * 7} days"
            }

        return test_recommendations

    def _calculate_sample_size(self, num_variations: int) -> str:
        """Calculate recommended sample size for A/B testing"""
        base_size = 1000 * num_variations
        return f"{base_size} users per variation"

    def generate_complete_campaign(self, brief: CampaignBrief, num_variations: int = 5) -> Dict:
        """Orchestrate the complete campaign generation process"""

        print("\n" + "🚀" * 20)
        print("STARTING CAMPAIGN GENERATION...")
        print("🚀" * 20)

        print("\n🔧 Step 1: Generating ad copies...")
        ad_copies = self.generate_ad_copies(brief, num_variations)
        time.sleep(1)

        print("🎨 Step 2: Creating image prompts...")
        image_prompts = self.create_image_prompts(ad_copies)
        time.sleep(1)

        print("🖼️  Step 3: Generating images...")
        generated_images = self.generate_images(image_prompts)
        time.sleep(1)

        # Create variations
        variations = []
        for copy, prompt, image in zip(ad_copies, image_prompts, generated_images):
            variations.append(AdVariation(copy=copy, image_prompt=prompt, generated_image=image))

        print("📊 Step 4: Evaluating coherence...")
        ranked_variations = self.evaluate_coherence(variations)
        time.sleep(1)

        print("🧪 Step 5: Creating A/B test plan...")
        ab_test_plan = self.create_ab_test_recommendations(ranked_variations)

        return {
            'campaign_brief': brief,
            'all_variations': ranked_variations,
            'top_combinations': ranked_variations[:3],
            'ab_test_recommendations': ab_test_plan,
            'summary': {
                'total_variations': len(ranked_variations),
                'average_coherence_score': sum(v.coherence_score for v in ranked_variations) / len(ranked_variations),
                'recommended_tests': len(ab_test_plan),
                'generation_mode': self.mode.value
            }
        }

def display_campaign_results(results: Dict):
    """Display the campaign results in a formatted way"""

    print("\n" + "="*60)
    print("🎯 CAMPAIGN GENERATION COMPLETE!")
    print("="*60)

    brief = results['campaign_brief']
    summary = results['summary']

    print(f"\n📋 CAMPAIGN BRIEF:")
    print(f"   Product: {brief.product_name}")
    print(f"   Target: {brief.target_audience}")
    print(f"   Goal: {brief.campaign_goal}")
    print(f"   Mode: {summary['generation_mode'].upper()}")

    print(f"\n📊 SUMMARY:")
    print(f"   Generated {summary['total_variations']} variations")
    print(f"   Average coherence score: {summary['average_coherence_score']:.1f}/10")
    print(f"   Recommended A/B tests: {summary['recommended_tests']}")

    print("\n🏆 TOP 3 COMBINATIONS:")
    for i, variation in enumerate(results['top_combinations'], 1):
        print(f"\n   {i}. Score: {variation.coherence_score:.1f}/10")
        print(f"      Headline: {variation.copy.headline}")
        print(f"      Body: {variation.copy.body}")
        print(f"      CTA: {variation.copy.call_to_action}")
        print(f"      Tone: {variation.copy.tone}")
        print(f"      Segment: {variation.copy.target_segment}")
        print(f"      Image: {variation.generated_image}")
        print(f"      Prompt: {variation.image_prompt[:80]}...")

    print("\n🧪 A/B TESTING RECOMMENDATIONS:")
    for segment, plan in results['ab_test_recommendations'].items():
        print(f"\n   👥 Segment: {segment}")
        print(f"      Priority: {plan['testing_priority']}")
        print(f"      Sample Size: {plan['sample_size_recommendation']}")
        print(f"      Duration: {plan['recommended_duration']}")
        print(f"      Variations to test: {len(plan['variations'])}")
        print(f"      Key Metrics: {', '.join(plan['key_metrics'])}")

def save_results(results: Dict):
    """Save results to JSON file"""
    output_file = "campaign_results.json"

    # Convert dataclasses to dictionaries for JSON serialization
    results_dict = {
        'campaign_brief': {
            'product_name': results['campaign_brief'].product_name,
            'target_audience': results['campaign_brief'].target_audience,
            'key_features': results['campaign_brief'].key_features,
            'brand_voice': results['campaign_brief'].brand_voice,
            'campaign_goal': results['campaign_brief'].campaign_goal,
            'competitors': results['campaign_brief'].competitors
        },
        'top_combinations': [
            {
                'coherence_score': var.coherence_score,
                'copy': {
                    'headline': var.copy.headline,
                    'body': var.copy.body,
                    'call_to_action': var.copy.call_to_action,
                    'tone': var.copy.tone,
                    'target_segment': var.copy.target_segment
                },
                'image_prompt': var.image_prompt,
                'generated_image': var.generated_image
            }
            for var in results['top_combinations']
        ],
        'summary': results['summary']
    }

    with open(output_file, 'w') as f:
        json.dump(results_dict, f, indent=2)

    print(f"\n💾 Results saved to {output_file}")

def main():
    """Main function with interactive user input"""
    print("🎪 WELCOME TO AI AD CAMPAIGN GENERATOR!")
    print("Create professional advertising campaigns in minutes!\n")

    # Get all inputs from user
    input_handler = UserInputHandler()

    # Step 1: Get campaign brief
    campaign_brief = input_handler.get_campaign_brief()

    # Step 2: Get generation settings
    num_variations, mode = input_handler.get_generation_settings()

    # Step 3: Initialize generator
    try:
        if mode == GenerationMode.OPENAI:
            api_key = input_handler.get_openai_key()
            generator = AdCampaignGenerator(openai_api_key=api_key, mode=mode)
        else:
            generator = AdCampaignGenerator(mode=mode)
    except Exception as e:
        print(f"❌ Error initializing generator: {e}")
        print("🔄 Using MOCK mode instead")
        generator = AdCampaignGenerator(mode=GenerationMode.MOCK)

    # Step 4: Generate campaign
    campaign_results = generator.generate_complete_campaign(campaign_brief, num_variations)

    # Step 5: Display results
    display_campaign_results(campaign_results)

    # Step 6: Save results
    save_results(campaign_results)

    print("\n🎉 Campaign generation completed successfully!")
    print("📈 You're now ready to run your A/B tests!")

if __name__ == "__main__":
    main()



🎪 WELCOME TO AI AD CAMPAIGN GENERATOR!
Create professional advertising campaigns in minutes!


📝 AD CAMPAIGN BRIEF CREATION

🎯 Product Name: LearnSmart
👥 Target Audience (e.g., 'Young professionals 25-35'): 18-25

🔑 Key Features (Enter one per line, press Enter twice to finish):
  → Feature: Interactive video lessons
  → Feature: Practice quizzes
  → Feature: Progress analytics
  → Feature: 

🎭 Brand Voice (e.g., 'Professional, Friendly, Innovative'): Innovative

🎯 Campaign Goal (e.g., 'Increase sales by 25%'): Acquire 20,000 premium subscribers

🏢 Competitors (Enter one per line, press Enter twice to skip):
  → Competitor: Corsera, udemy
  → Competitor: 

⚙️  GENERATION SETTINGS

🔢 How many ad variations to generate? (3-10): 5

🤖 Choose Generation Mode:
1. MOCK Mode (Fast, sample data - No API keys needed)
2. OPENAI Mode (Real AI - Requires OpenAI API key)

🎛️  Enter your choice (1 or 2): 1
🎭 Running in MOCK mode - generating sample data without API calls

🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
STARTIN